# LCA of various NG suppliers

In [ ]:
import os
import pprint
import yaml
from yaml.loader import SafeLoader
import warnings

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import brightway2 as bw

warnings.filterwarnings("ignore")

pprinter = pprint.PrettyPrinter(indent=4).pprint
plt.rcParams["svg.fonttype"] = "path"

BW_PROJECT = "NG_2022crisis_LCA"
DB_NAME = "ecoinvent 3.9.1 cutoff, natural gas scenario EU27"

bw.projects.set_current(BW_PROJECT)  # Accessing the project
ei_ng_db = bw.Database(DB_NAME)

PATH2RESULTS = "./data/results/"
if os.path.exists(PATH2RESULTS) is False:
    os.makedirs(PATH2RESULTS)
    
SAVE_FIGS = False

## Read activities

In [ ]:
with open("./data/ng_suppliers.yaml") as f:
    data = yaml.load(f, Loader=SafeLoader)["ng_suppliers"]
    print(data)

## Find activities in the DB

In [ ]:
ng_activities = [
    act for act in ei_ng_db if act["name"] in data and act["location"] == "EU27"
]
print(ng_activities)

## Select LCIA methods

In [ ]:
ipcc_method = [
    met
    for met in list(bw.methods)
    if met[0] == "IPCC 2021"
    and "climate change: including SLCFs" in met[1]
    and "GWP100" in met[2]
]
print(ipcc_method)

## Run LCA

In [ ]:
def do_lca(activities, methods, verbose=True):
    lca_results = {}

    for i, act in enumerate(activities):
        functional_unit = {act: 1.0e9}  # 1 bcm

        lca = bw.LCA(functional_unit)
        lca.lci()

        for met in methods:
            lca.switch_method(met)
            lca.lcia()
            if verbose:
                print(f"{act['name']}:\t {lca.score/1}")  # kg CO2 eq./year')
            lca_results[act["name"]] = lca.score

    return lca_results


lca_results = do_lca(ng_activities, ipcc_method, verbose=False)

df = pd.Series(lca_results)
df.sort_values(ascending=True, inplace=True)
df

In [ ]:
df.plot.barh()

plt.show()

## Process contribution

Run in activity browser the following LCA calculations:
 - Functional unit: 1 cubic meter of natural gas imported in gaseous form or LNG for various suppliers as in `./data/ng_suppliers.yaml` in location `EU27` from the new `ecoinvent 3.9.1 cutoff, natural gas scenario EU27` database
 - LCIA method: IPCC 2021, climate change: including SLCFs, global warming potential (GWP100) [kg CO2-Eq]
 - In `Process Contributions`: select aggregation method by reference product (limited to top 5 contributing activities)
 - Export absolute results in `"./data/results/ng_suppliers_process_contributions.csv"`


In [ ]:
def read_and_prepare_csv(path):
    try:
        df_proc_contrib = pd.read_csv(path, index_col=0)
    except FileNotFoundError:
        print(f"File {path} not found. Run the analysis described above first.")
        return None
    # df_proc_contrib.set_index('index')
    column_names = df_proc_contrib["index"].tolist()

    df_proc_contrib = df_proc_contrib.T
    df_proc_contrib.columns = column_names
    df_proc_contrib.drop("index", axis=0, inplace=True)
    df_proc_contrib["Rest"] = 2 * df_proc_contrib.Total - df_proc_contrib.sum(axis=1)
    df_proc_contrib.sort_values(by="Total", ascending=True, inplace=True)
    df_proc_contrib.drop("Total", axis=1, inplace=True)

    new_index_names = [
        ind.split(" | ")[1]
        .replace("natural gas, high pressure, ", "")
        .replace("import", "Natural gas import")
        .replace("regasification LNG", "LNG import")
        .replace("domestic", "Domestic")
        .replace("NL", "the Netherlands")
        .replace("NO", "Norway")
        .replace("DE", "Germany")
        .replace("RO", "Romania")
        .replace("QA", "Qatar")
        .replace("AZ", "Azerbaijan")
        .replace("US", "the US")
        .replace("RU", "Russia")
        .replace("DZ", "Algeria")
        .replace("from", "\nfrom")
        .replace(" in ", "\nin ")
        for ind in df_proc_contrib.index
    ]
    df_proc_contrib.rename(
        index={
            ind: new_ind for ind, new_ind in zip(df_proc_contrib.index, new_index_names)
        },
        inplace=True,
    )

    return df_proc_contrib

df_proc_contrib = read_and_prepare_csv(f"{PATH2RESULTS}ng_suppliers_process_contributions.csv") * 1e9

In [ ]:
fig, ax = plt.subplots(
    figsize=(9.72441, 7.20472),
    dpi=300 if SAVE_FIGS else 100,
)


PALLETE = [
    "#a50026",
    "#d73027",
    "#f46d43",
    "#fdae61",
    "#fee090",
    "#ffffbf",
    "#e0f3f8",
    "#abd9e9",
    "#74add1",
    "#4575b4",
    "#313695",
]
sns.set_style("ticks")  # darkgrid, white grid, dark, white and ticks
plt.rcParams["font.family"] = "arial"
plt.rcParams["legend.facecolor"] = "white"
plt.rcParams["legend.edgecolor"] = "black"
plt.rcParams["legend.shadow"] = False
plt.rcParams["legend.frameon"] = True
plt.rcParams["patch.linewidth"] = 0.3
plt.rcParams["font.size"] = 8
plt.rcParams["axes.titlesize"] = 8
plt.rcParams["axes.titlepad"] = 3.0
plt.rcParams["axes.labelsize"] = 8
plt.rcParams["axes.ymargin"] = 0.0
plt.rcParams["axes.xmargin"] = 0.05
plt.rcParams["axes.edgecolor"] = "black"
plt.rcParams["axes.linewidth"] = 0.3
plt.rcParams["axes.grid"] = True
plt.rcParams["grid.linewidth"] = 0.4
plt.rcParams["axes.spines.right"] = True
plt.rcParams["axes.spines.top"] = True

df_proc_contrib.plot.barh(
    stacked=True, ax=ax, color=PALLETE, alpha=1.0, edgecolor="black", linewidth=0.3
)

ax.set_xlabel("IPCC 2021, climate change, GWP100 [kg CO$_2$-eq.]")
ax.xaxis.set_ticks([i * 1e8 for i in range(10)])
handles, labels = ax.get_legend_handles_labels()

plt.legend(
    handles,
    [lab.capitalize() for lab in labels],
    ncol=4,
    loc="lower right",
    bbox_to_anchor=(1.04, -0.18),
)

if SAVE_FIGS:
    plt.savefig("./figs/ng_suppliers_proc_contrib.png", bbox_inches="tight")
    plt.savefig("./figs/ng_suppliers_proc_contrib.svg", bbox_inches="tight")

## Elementary flow contibutions

Run in activity browser the following LCA calculations:
 - Functional unit: 1 cubic meter of natural gas imported in gaseous form or LNG for various suppliers as in `./data/ng_suppliers.yaml` in location `EU27` from the new `ecoinvent 3.9.1 cutoff, natural gas scenario EU27` database
 - LCIA method: IPCC 2021, climate change: including SLCFs, global warming potential (GWP100) [kg CO2-Eq]
 - In `EF Contributions`: select aggregation method by name (limited to top 3 contributing flows)
 - Export Elementary Flow Contribution absolute results in `"./data/results/ng_suppliers_elementary_flow_contributions.csv"`

In [ ]:
df_ef_contrib = (
    read_and_prepare_csv(
        f"{PATH2RESULTS}ng_suppliers_elementary_flow_contributions.csv"
    )
    * 1e9
)
df_ef_contrib["Rest"] = df_ef_contrib[
    [
        col
        for col in df_ef_contrib.columns
        if col
        not in {"Methane, fossil", "Carbon dioxide, fossil", "Carbon monoxide, fossil"}
    ]
].sum()

df_ef_contrib = df_ef_contrib[
    ["Methane, fossil", "Carbon dioxide, fossil", "Carbon monoxide, fossil", "Rest"]
]

In [ ]:
fig, ax = plt.subplots(
    figsize=(9.72441, 7.20472),
    dpi=300 if SAVE_FIGS else 100,
)

PALLETE = ["#d7191c", "#fdae61", "#abd9e9", "#2c7bb6"]

df_ef_contrib.plot.barh(
    stacked=True, ax=ax, color=PALLETE, alpha=1.0, edgecolor="black", linewidth=0.3
)

ax.set_xlabel("IPCC 2021, climate change, GWP100 [kg CO$_2$-eq.]")
ax.xaxis.set_ticks([i * 1e8 for i in range(10)])
handles, labels = ax.get_legend_handles_labels()

plt.legend(
    handles,
    [lab.capitalize() for lab in labels],
    ncol=4,
    loc="lower right",
    bbox_to_anchor=(0.85, -0.12),
)

if SAVE_FIGS:
    plt.savefig("./figs/ng_suppliers_ef_contrib.png", bbox_inches="tight")
    plt.savefig("./figs/ng_suppliers_ef_contrib.svg", bbox_inches="tight")

# Country dependence on Russian gas (2021)

In [ ]:
with open("./data/eu_members.yaml") as f:
    eu_members = yaml.load(f, Loader=SafeLoader)["eu_members"]

df_imports = pd.read_excel(
    "./data/ng-supply-demand-raw/nrg_ti_gas_eurostat.xlsx",
    sheet_name="Sheet 1",
    header=9,
)
df_imports.dropna(axis=0, inplace=True)
df_imports.replace(":", 0, inplace=True)
df_imports.columns = ["Importer", "Russia", "Total"]
df_imports.set_index("Importer", drop=True, inplace=True)
df_imports.rename(
    index={"European Union - 27 countries (from 2020)": "EU27"}, inplace=True
)
df_imports = df_imports.loc[["EU27"] + eu_members] / 1000
df_imports.sort_values("Total", ascending=False, inplace=True)
df_imports["Total"] = df_imports["Total"] - df_imports["Russia"]
df_imports.head()

In [ ]:
fig, ax = plt.subplots(figsize=(9.72441, 7.20472), dpi=300 if SAVE_FIGS else 100)

PALLETE = [
    "#4daf4a",
    "#984ea3",
    "#ff7f00",
]

df_imports.plot.barh(
    stacked=True,
    ax=ax,
    color=PALLETE,
    alpha=1.0,
    edgecolor="black",
    linewidth=0.3,
)

ax.set_xlabel("Natural gas imports in 2021 [bcm year$^{-1}$]")
handles, labels = ax.get_legend_handles_labels()

plt.legend(
    handles,
    [lab.capitalize() for lab in labels],
    ncol=4,
    loc="upper right",
    title="Suppliers",
    # bbox_to_anchor=(0.85, -0.12),
)

if SAVE_FIGS:
    plt.savefig("./figs/ng_import_by_country.png", bbox_inches="tight")
    plt.savefig("./figs/ng_import_by_country.svg", bbox_inches="tight")